In [30]:
from IPython.core.display import display,HTML
display(HTML("<style>.container {width:95% !important;} </style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Tutorial IBS

In [2]:
import IBSLib as ibslib
import numpy as np
import pandas as pd

## Physical Constants


In [3]:
print("{:35} : {:12.8e}".format("Speed of light in m/s", ibslib.clight))
print("{:35} : {:12.8e}".format("Reduced Planck constant in GeV ", ibslib.hbarGeV))
print("{:35} : {:12.8e}".format("Electron Mass in GeV", ibslib.electron_mass))
print("{:35} : {:12.8e}".format("Proton Mass in GeV", ibslib.proton_mass))
print("{:35} : {:12.8e}".format("Neutron Mass in GeV", ibslib.neutron_mass))
print("{:35} : {:12.8e}".format("Muon Mass in GeV", ibslib.mu_mass))
print("{:35} : {:12.8e}".format("Atomic Mass Unit in GeV", ibslib.atomic_mass_unit))
print("{:35} : {:12.8e}".format("Pi", ibslib.pi))
print("{:35} : {:12.8e}".format("Electric Charge in Coulomb", ibslib.electric_charge))
print("{:35} : {:12.8e}".format("Euler Constant", ibslib.euler))
print("{:35} : {:12.8e}".format("Classical Electron Radius in m", ibslib.electron_radius))
print("{:35} : {:12.8e}".format("Classical Proton Radius in m ", ibslib.proton_radius))


Speed of light in m/s               : 2.99792458e+08
Reduced Planck constant in GeV      : 6.58211957e-25
Electron Mass in GeV                : 5.10998950e-04
Proton Mass in GeV                  : 9.38272088e-01
Neutron Mass in GeV                 : 9.39565421e-01
Muon Mass in GeV                    : 1.05658375e-01
Atomic Mass Unit in GeV             : 9.31494102e-01
Pi                                  : 3.14159265e+00
Electric Charge in Coulomb          : 1.60217663e-19
Euler Constant                      : 5.77215665e-01
Classical Electron Radius in m      : 2.81794033e-15
Classical Proton Radius in m        : 1.53469827e-18


## Twiss

- Cpymad
- from file
    * read header
    * read table
    * update -> adds radiation integrals per element (I1,I2,I3,I4x,I4y,I5x,I5y)

### Cpymad

In [4]:
from cpymad.madx import Madx

lattice = "BII_1996-01-01_design_v_1.seq"
tfsfile = "b2_design.twiss"

mad = Madx()
mad.option.echo = False
mad.option.warn = False
mad.command.beam(energy=1.7, particle='electron')
mad.call(file=lattice)
mad.use(sequence='ring')
twiss = mad.twiss(file=tfsfile, chrom=True)
twiss.summary

{'sequence': 'RING', 'particle': 'ELECTRON', 'mass': 0.00051099895, 'charge': -1.0, 'energy': 1.7, 'pc': 1.699999923, 'gamma': 3326.817012, 'kbunch': 1.0, 'bcurrent': 2.001335116e-13, 'sige': 0.001, 'sigt': 1.0, 'npart': 1.0, 'ex': 1.0, 'ey': 1.0, 'et': 0.001, 'bv_flag': 1.0, 'length': 240.0, 'alfa': 0.0007316942746, 'orbit5': -0.0, 'gammatr': 36.9687845, 'q1': 17.84996563, 'q2': 6.743031469, 'dq1': 1.077745266, 'dq2': 1.371772348, 'dxmax': 0.4496533372, 'dymax': 0.0, 'xcomax': 0.0, 'ycomax': 0.0, 'betxmax': 17.43722837, 'betymax': 21.09492526, 'xcorms': 0.0, 'ycorms': 0.0, 'dxrms': 0.2272073799, 'dyrms': 0.0, 'deltap': 0.0, 'synch_1': 0.1756066259, 'synch_2': 1.442924619, 'synch_3': 0.3313655981, 'synch_4': -0.004653715528, 'synch_5': 0.001749753833}

###  From File

In [5]:
# read twiss summary / header as dictionary
twissHeader = ibslib.GetTwissHeader(tfsfile)
twissHeader.keys()

dict_keys(['ALFA', 'BCURRENT', 'BETXMAX', 'BETYMAX', 'BV_FLAG', 'CHARGE', 'DELTAP', 'DQ1', 'DQ2', 'DXMAX', 'DXRMS', 'DYMAX', 'DYRMS', 'ENERGY', 'ET', 'EX', 'EY', 'GAMMA', 'GAMMATR', 'KBUNCH', 'LENGTH', 'MASS', 'NPART', 'ORBIT5', 'PC', 'Q1', 'Q2', 'SIGE', 'SIGT', 'SYNCH_1', 'SYNCH_2', 'SYNCH_3', 'SYNCH_4', 'SYNCH_5', 'XCOMAX', 'XCORMS', 'YCOMAX', 'YCORMS'])

In [6]:
# read twiss table as dictionary - only reads IBS relevant columns from Twiss (data reduction)
twissTable = ibslib.GetTwissTable(tfsfile)
twissTable.keys()

dict_keys(['ALFX', 'ALFY', 'ANGLE', 'BETX', 'BETY', 'DPX', 'DPY', 'DX', 'DY', 'K1L', 'K1SL', 'L'])

In [7]:
# update Twiss with radiation integrals and related quantities needed later on (come back to this later)
twissTable = ibslib.updateTwiss(twissTable)
twissTable.keys()

dict_keys(['ALFX', 'ALFY', 'ANGLE', 'BETX', 'BETY', 'DPX', 'DPY', 'DX', 'DY', 'I1', 'I2', 'I3', 'I4x', 'I4y', 'I5x', 'I5y', 'K1L', 'K1SL', 'L', 'gammax', 'gammay', 'hx', 'hy', 'k', 'rho'])

In [8]:
# print values of twiss column
ibslib.printTwissColumn("BETX",twissTable)

16.8602
17.3272
17.382
17.4372
12.1437
9.41272
6.93308
4.92173
2.80142
0.988134
0.803841
2.4349
3.85878
7.29053
9.52229
14.3048
16.6995
16.6778
16.6653
16.654
16.6394
14.2024
9.40131
7.16668
3.74428
2.32885
0.72913
1.10602
3.07841
5.48473
7.86307
10.8148
14.0817
15.5822
12.9317
10.4275
8.28888
6.58412
6.58412
8.28888
10.4275
12.9317
15.5822
14.0817
10.8148
7.86307
5.48473
3.07841
1.10602
0.72913
2.32885
3.74428
7.16668
9.40131
14.2024
16.6394
16.654
16.6653
16.6778
16.6995
14.3048
9.52229
7.29053
3.85878
2.4349
0.803841
0.988134
2.80142
4.92173
6.93308
9.41272
12.1437
17.4372
17.382
17.3272
17.3272
17.382
17.4372
12.1437
9.41272
6.93308
4.92173
2.80142
0.988134
0.803841
2.4349
3.85878
7.29053
9.52229
14.3048
16.6995
16.6778
16.6653
16.654
16.6394
14.2024
9.40131
7.16668
3.74428
2.32885
0.72913
1.10602
3.07841
5.48473
7.86307
10.8148
14.0817
15.5822
12.9317
10.4275
8.28888
6.58412
6.58412
8.28888
10.4275
12.9317
15.5822
14.0817
10.8148
7.86307
5.48473
3.07841
1.10602
0.72913
2.32885
3.7

## Radiation Damping
- RadiationDampingApprox
- RadiationDampingLattice
- RadiationDampingEquilibria

###  Radiation Damping Approx

Uses ring averages (Tunes -> beta average, average bending radius, etc...)

In [9]:
df     = pd.DataFrame.from_dict(twissTable,orient='columns')
rhoAvg = df.loc[df.rho != 0.0,'rho'].mean()

gamma    = twissHeader['GAMMA']
gamma_tr = twissHeader['GAMMATR']
circ     = twissHeader['LENGTH']

bxAvg = circ / (2.0 * ibslib.pi * twissHeader["Q1"]); 
byAvg = circ / (2.0 * ibslib.pi * twissHeader["Q2"]);

radint = np.zeros(7)
ibslib.RadiationDampingApprox(circ, gamma, gamma_tr, rhoAvg, bxAvg, byAvg, radint);
radint

array([1.540409  , 1.44292462, 0.3313656 , 0.        , 0.        ,
       0.07523688, 0.        ])

### Radiation Damping Lattice

Uses Twiss Table

In [10]:
ibslib.RadiationDampingLattice(twissTable, radint)
radint

array([ 0.26299979,  1.44292462,  0.3313656 , -0.00465372,  0.        ,
        0.00201665,  0.        ])

### Radiation damping Equilibria

Values are used in the ODE simulations!!

In [11]:
gamma   = twissHeader.get('GAMMA')
pc      = twissHeader.get('PC')
gammatr = twissHeader.get('GAMMATR')
mass    = twissHeader.get('MASS')
charge  = twissHeader.get('CHARGE')
q1      = twissHeader.get('Q1')
l       = twissHeader.get('LENGTH')

aatom   = ibslib.electron_mass / ibslib.proton_mass
betar   = ibslib.beta_relativistic_from_gamma(gamma) # see numeric functions
r0      = ibslib.particle_radius(1, aatom) # see numeric functions
trev    = l / (betar * ibslib.clight)
frev    = 1.0 / trev
omega   = 2.0 * ibslib.pi * frev
eta     = ibslib.eta(gamma, gammatr)
epsilon = 1.0e-6 # accuracy for synchronuous phase search

radint = np.zeros(7)
ibslib.RadiationDampingLattice(twissTable, radint)

target_energy_gain_ev    = 0.0
intial_phis_search_value = 173
harmon                   = [400.0]
voltages                 = [-4. * 375000.]

# methods below are described/ demonstrated in the numeric functions part
U0     = ibslib.get_energy_loss_per_turn(twissHeader, radint[1], aatom) 
phis   = ibslib.get_synchronuous_phase(target_energy_gain_ev, intial_phis_search_value, U0, charge, harmon,voltages,epsilon)
qs     = ibslib.get_synchrotron_tune(omega, U0, charge, harmon, voltages, phis, eta, pc)
omegas = qs * omega

equi = np.zeros(9)
ibslib.RadiationDampingEquilibria(twissHeader, radint, aatom, qs, equi)
print("{:10} : {:10.6e} ({})\n".format("Energy Loss per Turn", U0, "eV"));
print("{:10} : {:10.6e} ({})".format("Taux", equi[0], "s"));
print("{:10} : {:10.6e} ({})".format("Tauy", equi[1], "s"));
print("{:10} : {:10.6e} ({})".format("Taus", equi[2], "s"));
print("{:10} : {:10.6e} ({})".format("exinf", equi[3], ""));
print("{:10} : {:10.6e} ({})".format("eyinf", equi[4], ""));
print("{:10} : {:10.6e} ({})".format("sigeoe2", equi[5], ""));
print("{:10} : {:10.6e} ({})".format("sigtinf", equi[6], ""));
print("{:10} : {:10.6e} ({})".format("jx", equi[7], ""));
print("{:10} : {:10.6e} ({})".format("jy", equi[8], ""))

Energy Loss per Turn : 1.702619e+05 (eV)

Taux       : 7.967527e-03 (s)
Tauy       : 7.993224e-03 (s)
Taus       : 3.990177e-03 (s)
exinf      : 5.908327e-09 ()
eyinf      : 9.415563e-14 ()
sigeoe2    : 4.877655e-07 ()
sigtinf    : 3.054559e-03 ()
jx         : 1.003225e+00 ()
jy         : 1.000000e+00 ()


## Numeric Functions

- Standard util functions
- Longitudinal / RF
- IBS specific numeric functions

### Standard functions

In [12]:
betar   = ibslib.beta_relativistic_from_gamma(gamma)
eta     = ibslib.eta(gamma, gammatr)
r0      = ibslib.particle_radius(1, aatom)

sige    = np.sqrt(equi[5])
dpop    = ibslib.dee_to_dpp(sige, betar)
deoe    = ibslib.dpp_to_dee(dpop, betar)
betar, eta, r0, sige, dpop, deoe

(0.999999954823541,
 0.0007316942747206528,
 2.8179403262e-15,
 0.0006984020902062896,
 0.0006984021532869811,
 0.0006984020902063293)

### Longitudinal / RF
- sigs_from_sige
- sige_from_sigs
- rf_voltage_in_eV - energy gain in eV -> to be able to find $\phi_s$
- rf_voltage_in_ev_prime - derivative of rf_voltage_in_eV 
- rf_voltage_in_ev_with_rad_losses
- get_synchronuous_phase
- rf_voltage_with_potential_well_distortion
- rf_voltage_with_potential_well_distortion_prime
- get_synchronuous_phase_with_potential_well_distortion
- get_synchrotron_tune
- get_synchrotron_tune_with_potential_well
- sige_form_sigs_using_rf

In [14]:
# sigma_s from sigma_e
sigs = ibslib.sigs_from_sige(sige, gamma, gammatr, omegas)
sigs, sige

(0.003054558934794126, 0.0006984020902062896)

In [15]:
# sigma_e from sigma_s
sigen = ibslib.sige_from_sigs(omega, sigs, qs, gamma, gammatr)
sigs,sigen

(0.003054558934794126, 0.0006984020902062895)

In [21]:
phi = 173
ibslib.rf_voltage_in_ev(phi, charge, harmon,voltages), ibslib.rf_voltage_in_ev_prime(phi, charge, harmon,voltages)

(182804.01510772132, -1488819.227461983)

In [22]:
# offset to be on synchronuous phase
ibslib.rf_voltage_in_ev_with_rad_losses(phi, charge, U0, harmon, voltages)

12542.106380552286

In [24]:
# synchronuous phase
phis   = ibslib.get_synchronuous_phase(target_energy_gain_ev, intial_phis_search_value, U0, charge, harmon,voltages,epsilon)
intial_phis_search_value, phis, ibslib.rf_voltage_in_ev_with_rad_losses(phis, charge, U0, harmon, voltages)

(173, 173.48242725610714, 9.92469722405076e-07)

In [26]:
# synchrotron tune 
ibslib.get_synchrotron_tune(omega, U0, charge, harmon, voltages, phis, eta, pc)

0.006390253850181927

In [29]:
ibslib.sige_form_sigs_using_rf(sigs, U0, charge, harmon, voltages, gamma,gammatr, pc, circ, phis, True), sige

(0.0006984021533089606, 0.0006984020902062896)

### IBS specific

In [32]:
# numeric functions used in IBS growth rate calculations
ibslib.fmohl(1.,2.,3.,1000), ibslib.rds(1.0,2.0,3.0)

(-5.404172967883226, 0.29046028102188937)

## Coulomb Log
- twclog
- twclogtail
- coulomblog
- coulomblogtail

The first two methods are used in the simulation and are designed to be applied on an element by element basis. 
The second two use ring averages and can be used to get an idea of the values.

In [41]:
pnumber = 1.0e10
restE   = twissHeader['ENERGY']
clog    = np.zeros(2)

bx  = twissTable['BETX'][23]
by  = twissTable['BETY'][23]
dx  = twissTable['DX'][23]
dy  = twissTable['DY'][23]
ex  = 7.5e-9
ey  = ex * 0.05

ibslib.twclog(pnumber, bx, by, dx, dy, ex, ey, r0, gamma, charge, restE, mass, sige, sigs, clog )

print("{:25} : {:12.6e}".format("Coulomb Log",clog[0]))
print("{:25} : {:12.6e}".format("Coulomb IBS pre-factor",clog[1]))

Coulomb Log               : 1.896855e+01
Coulomb IBS pre-factor    : 2.444649e-02
